# Check Environment and Run Training

In [2]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO
from customenv import PharmacophoreEnv
from wandb.integration.sb3 import WandbCallback
import wandb
import time
from stable_baselines3.common.monitor import Monitor
import gymnasium as gym
from gymnasium.envs.registration import register
from gymnasium.wrappers import FlattenObservation
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv


In [4]:
# Instantiate the env
env = PharmacophoreEnv(output=r'C:\Users\kilia\MASTER\data\hitlists\hitlist', 
                       querys=r'C:\Users\kilia\MASTER\V2Z1551.pml', 
                       actives_db=r'C:\Users\kilia\MASTER\data\seh_actives_mini.ldb', 
                       inactives_db=r"C:\Users\kilia\MASTER\data\seh_inactives_mini.ldb",
                       ldba=36,
                       ldbi=112,
                       features="H,HBA,HBD")

# check custom environment and output additional warnings if needed
check_env(env)


c:\Users\kilia\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
# Example for the CartPole environment
register(
    # unique identifier for the env `name-version`
    id="PharmacophoreEnv-v0",
    # path to the class for creating the env
    entry_point="customenv:PharmacophoreEnv",
    # Max number of steps per episode, using a `TimeLimitWrapper`
    kwargs={"output":r'C:\Users\kilia\MASTER\data\hitlists\hitlist', 
    "querys":r'C:\Users\kilia\MASTER\V2Z1551.pml', 
    "actives_db":r'C:\Users\kilia\MASTER\data\seh_actives_mini.ldb', 
    "inactives_db":r"C:\Users\kilia\MASTER\data\seh_inactives_mini.ldb",
    "ldba":36,
    "ldbi":112,
    "features":"H,HBA,HBD"},
)

c:\Users\kilia\MASTER\data\venv\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment PharmacophoreEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [6]:
env = gym.make("PharmacophoreEnv-v0")
env = Monitor(env)
# env = FlattenObservation(env)
env.reset()
batch_size = 64
#env = make_vec_env(env_id=env, n_envs=6, vec_env_cls=SubprocVecEnv)

In [12]:
config = {"policy_type": "MultiInputPolicy", "total_timesteps": 2500}
experiment_name = f"PPO_{int(time.time())}"

#wandb.tensorboard.patch(root_logdir=f"runs/{experiment_name}")
#wandb.init(project="repharm", config=config, name=experiment_name, sync_tensorboard=True)
# Define and Train the agent
model = PPO(config["policy_type"], "PharmacophoreEnv-v0", verbose=2, tensorboard_log=f"runs/{experiment_name}", n_epochs=10)

model.learn(config["total_timesteps"], 
        #     callback=WandbCallback(gradient_save_freq=100,
        #                            model_save_freq=1000, 
        #                            model_save_path=f"models/{experiment_name}",
        #                            verbose=2
        # )
        )
model.save("models/finals")
#wandb.finish()

Using cpu device
Creating environment from the given name 'PharmacophoreEnv-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\kilia\MASTER\data\venv\lib\site-packages\gymnasium\envs\registration.py:491: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'features']
  logger.warn(
c:\Users\kilia\MASTER\data\venv\lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in runs/PPO_1683009037\PPO_1


Logging to runs/PPO_1683009037\PPO_1


KeyboardInterrupt: 

In [11]:
from stable_baselines3 import A2C

model = A2C("MlpPolicy", "CartPole-v1", verbose=2, tensorboard_log="./a2c_cartpole_tensorboard/")
model.learn(total_timesteps=10000)

wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in ./a2c_cartpole_tensorboard/A2C_2


Using cpu device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./a2c_cartpole_tensorboard/A2C_2
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 32.2     |
|    ep_rew_mean        | 32.2     |
| time/                 |          |
|    fps                | 1534     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.684   |
|    explained_variance | 0.0712   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.42     |
|    value_loss         | 7.58     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 37.5     |
|    ep_rew_mean        | 37.5     |
| time/                 |     